In [1]:
#import matplotlib.pyplot as plt
import mdtraj
import numpy as np

import alphaspace2 as al

In [4]:
bcl2_prot, bcl2_lig = mdtraj.load('Bcl2_BAX/prot.pdb'),mdtraj.load('Bcl2_BAX/lig.pdb')
bclxl_prot, bclxl_lig = mdtraj.load('BclXL_BAX/prot.pdb'),mdtraj.load('BclXL_BAX/lig.pdb')
al.annotateVinaAtomTypes(pdbqt="Bcl2_BAX/prot.pdbqt", receptor=bcl2_prot)
al.annotateVinaAtomTypes(pdbqt="BclXL_BAX/prot.pdbqt", receptor=bclxl_prot)

In [5]:
ss_bcl2 = al.Snapshot()
ss_bcl2.run(bcl2_prot, bcl2_lig)
ss_bclxl = al.Snapshot()
ss_bclxl.run(bclxl_prot, bclxl_lig)

In [6]:
centroid_pockets_to_write=[[],[]]
betas_pockets_to_write = [[],[]]
bcl2_bclxl = al.Trajectory(snapshots=[ss_bcl2,ss_bclxl])
bcl2_bclxl.gen_dpockets(clust_distance=4.7)
contact_pockets = []
dps = sorted([dp for dp in bcl2_bclxl.dpockets],key=lambda i:sum(i.scores))
for dpx,dp in enumerate(dps):
    pockets = list(dp.pockets)
    if pockets[0].isContact or pockets[1].isContact:
        contact_pockets.append(dpx)
        print(dpx,"\t",pockets[0].score,"\t",pockets[1].score)
        if pockets[0].score!=0:
            centroid_pockets_to_write[0].append((dpx,pockets[0].centroid))
            betas_pockets_to_write[0].append((dpx,[b.xyz for b in pockets[0].betas]))
        if pockets[1].score!=0:
            centroid_pockets_to_write[1].append((dpx,pockets[1].centroid))
            betas_pockets_to_write[1].append((dpx,[b.xyz for b in pockets[1].betas]))


0 	 -5.362207 	 -5.859367
2 	 -2.9221647 	 -5.562416
3 	 -5.4417605 	 -2.359567
8 	 -6.2184644 	 0
10 	 -4.6135807 	 -1.3341222
11 	 -4.6906605 	 -1.164552
29 	 -2.2272186 	 -1.4742478
42 	 -1.4756579 	 -1.4238719
43 	 -1.0994188 	 -1.7869568
54 	 -1.2893351 	 -0.97642815
56 	 -1.2229697 	 -0.9430386
59 	 0 	 -2.1016717
60 	 0 	 -2.0681152
85 	 -0.6299271 	 -0.73714805
89 	 0 	 -1.2241524
96 	 -0.37557366 	 -0.6389315
106 	 -0.69140965 	 0
123 	 0 	 -0.39476568
125 	 -0.360147 	 0


In [8]:
contact_beta_atoms =[np.array([]),np.array([])]
for bx,betas in enumerate(betas_pockets_to_write):
    for _,pock in betas:
        contact_beta_atoms[bx] = np.append(contact_beta_atoms[bx],pock)


In [9]:
contact_beta_atoms = [contact_beta_atoms[0].reshape(-1,3),contact_beta_atoms[1].reshape(-1,3)]   

In [19]:
from beta_usr import Get_USR_alpha_beta

In [14]:
def soergel(i,j):   ### Distance calculation (Soergel Distance)
    li=len(i)
    lj=len(j)
    if li is not lj:
        print('lengths not equal')
    else:
        ntemp=0
        dtemp=0
        for k in range(li):
            ntemp=ntemp+abs(i[k]-j[k])
            dtemp=dtemp+max(i[k],j[k])
        score=float(ntemp)/float(dtemp)
    return score
        

In [35]:
usr_1 = Get_USR_alpha_beta(contact_beta_atoms[0])
usr_2 = Get_USR_alpha_beta(contact_beta_atoms[1])

In [37]:
usr_1     ## USR features, ctd_i = ith Moment of Distance of all beta atoms to Centroid (ctd point)
          ## cst_i = ith Moment of Distance of all beta atoms to beta atom closest to Centroid (cst point)
          ## fct_i = ith Moment of Distance of all beta atoms to beta atom furthest to Centroid (fct point)
          ## fct_i = ith Moment of Distance of all beta atoms to beta atom furthest from fct point (ftf point)

{'ctd_1': 9.66,
 'ctd_2': 4.273,
 'ctd_3': 0.929,
 'cst_1': 9.956,
 'cst_2': 4.58,
 'cst_3': 0.751,
 'fct_1': 22.45,
 'fct_2': 9.565,
 'fct_3': -0.641,
 'ftf_1': 17.828,
 'ftf_2': 9.489,
 'ftf_3': 0.559}

In [16]:
usr_1 = [s for _,s in usr_1.items()]   ## convert USR dictionary into arrays
usr_2 = [s for _,s in usr_2.items()]

In [17]:
print(1-soergel(usr_1,usr_2))    ## similarity of USR features of beta atoms

0.9002270544057287


In [18]:
from alphaspace2.functions import getSASA

In [27]:
from collections import defaultdict
def get_types_dict(filename):
    types_dict = defaultdict(dict)
    fp = open(filename, 'r')
    lines = fp.readlines()
    fp.close()
    for line in lines:
        line = line.split()
        types_dict[line[2]][line[0]]=line

    return types_dict

atom_types=['P','N','D','A','DA','AR','HP','PL']
atom_types_match={'P':'Positive_OASA','N':'Negative_OASA','D':'H_bond_Donor_OASA','A':'H_bond_Acceptor_OASA',
                  'DA':'H_bond_Doneptor_OASA','AR':'Aromatic_OASA','HP':'Hydrophobic_OASA','PL':'Polar_OASA','NL':'Null_type_OASA'}
prot_dict=get_types_dict('amber_types_fine.dat')
res_names=prot_dict.keys()

In [28]:
def get_pharmacophore_fingerprint(prot_md,beta_atoms):
    s1 = getSASA(prot_md)
    s2 = getSASA(prot_md,cover_atom_coords = beta_atoms/10)
    diff_bool = s1-s2 > 0
    d_asa = (s1-s2)[diff_bool]*100
    topology = prot_md.topology
    table, bonds = topology.to_dataframe()
    occluded_asa = []
    for top,asa in zip(table.values[diff_bool],d_asa):
        if top[4] in res_names:
            if top[1] in prot_dict[top[4]].keys():
                occluded_asa.append(tuple([prot_dict[top[4]][top[1]][-1],asa]))
            else:
                occluded_asa.append(tuple(['NL',asa]))
        else:
            occluded_asa.append(tuple(['NL',asa]))
    temp_surf_dict={'Total_OASA':0,'Positive_OASA':0,'Negative_OASA':0,'H_bond_Donor_OASA':0,'H_bond_Acceptor_OASA':0,
                    'H_bond_Doneptor_OASA':0,'Aromatic_OASA':0,'Hydrophobic_OASA':0,'Polar_OASA':0,'Null_type_OASA':0}
    for dd in occluded_asa:
        temp_surf_dict['Total_OASA']+=dd[1]
        temp_surf_dict[atom_types_match[dd[0]]] += dd[1]
    return temp_surf_dict


In [33]:
oasa_1 = get_pharmacophore_fingerprint(bcl2_prot,contact_beta_atoms[0])
oasa_2 = get_pharmacophore_fingerprint(bclxl_prot,contact_beta_atoms[1])

In [34]:
oasa_2    ## Dictionary of the Surface Area properties

{'Total_OASA': 872.6194033697248,
 'Positive_OASA': 17.201237201690674,
 'Negative_OASA': 75.00213193893433,
 'H_bond_Donor_OASA': 16.06208124011755,
 'H_bond_Acceptor_OASA': 173.10759711265564,
 'H_bond_Doneptor_OASA': 46.429887771606445,
 'Aromatic_OASA': 142.39950075745583,
 'Hydrophobic_OASA': 366.94288751482964,
 'Polar_OASA': 35.474079832434654,
 'Null_type_OASA': 0}

In [31]:
oasa_1 = [s for _,s in oasa_1.items()]    ## convert surface area dictionary into arrays
oasa_2 = [s for _,s in oasa_2.items()]

In [32]:
print(1-soergel(oasa_1,oasa_2))    ### similarity of Pharmacophore Occluded Surface Area, Score of 1 is perfect similarity 0 is most dissimilar

0.8164302738534024
